In [1]:
import pandas as pd

#Convert accepted_loans data to Panda DataFrame
accepted_df = pd.read_csv("/Users/abubakaral-fak i/Documents/Data Project/MPV1/data/raw/accepted_2007_to_2018Q4.csv")

/var/folders/yj/v54tzbn14gb70ts6s0j5s3xm0000gn/T/ipykernel_94520/3863113262.py:4: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  accepted_df = pd.read_csv("/Users/abubakaral-faki/Documents/Data Project/MPV1/data/raw/accepted_2007_to_2018Q4.csv")


In [2]:
#create a copy of data frame to work with temporarily before applying changes to original 'accepted_df' DataFrame
accepted_df_copy = pd.DataFrame(accepted_df)

print(accepted_df_copy.head())

         id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  68407277        NaN     3600.0       3600.0           3600.0   36 months   
1  68355089        NaN    24700.0      24700.0          24700.0   36 months   
2  68341763        NaN    20000.0      20000.0          20000.0   60 months   
3  66310712        NaN    35000.0      35000.0          35000.0   60 months   
4  68476807        NaN    10400.0      10400.0          10400.0   60 months   

   int_rate  installment grade sub_grade  ... hardship_payoff_balance_amount  \
0     13.99       123.03     C        C4  ...                            NaN   
1     11.99       820.28     C        C1  ...                            NaN   
2     10.78       432.66     B        B4  ...                            NaN   
3     14.85       829.90     C        C5  ...                            NaN   
4     22.45       289.91     F        F1  ...                            NaN   

  hardship_last_payment_amount disbursement_

In [3]:
output_path = '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/accepted_df_copy.csv'

accepted_df_copy.head(10000).to_csv(output_path, index = False)

### Create a CSV of each column and it's datatype

In [4]:
# Get data types of each variable
variable_data_types_df = accepted_df.dtypes.reset_index()

# Rename columns
variable_data_types_df.columns = ['Variables', 'Data Type']
print(variable_data_types_df)

#Save as csv
try:
    file_path = '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/variables_data_type.csv'
    
    variable_data_types_df.to_csv(file_path, index = False)
    print("Successfully saved variable_data_types_df as a csv.")

except Exception as e:
    print(e)

                 Variables Data Type
0                       id    object
1                member_id   float64
2                loan_amnt   float64
3              funded_amnt   float64
4          funded_amnt_inv   float64
..                     ...       ...
146      settlement_status    object
147        settlement_date    object
148      settlement_amount   float64
149  settlement_percentage   float64
150        settlement_term   float64

[151 rows x 2 columns]
Successfully saved variable_data_types_df as a csv.


### Convert Date columns to Date Data Types

In [5]:
# Convert Date_columns to Date Data types
date_columns = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'next_pymnt_d',
                'last_credit_pull_d', 'sec_app_earliest_cr_line', 'hardship_start_date',
                'hardship_end_date', 'payment_plan_start_date', 'debt_settlement_flag_date', 'settlement_date']

print(len(date_columns))

11


In [6]:
# Check format of date before converting to date objects

# We use dropna to drop all missing values in a columns because sometimes the first few rows have missing values

accepted_df.settlement_date.dropna().head() # Date format "%b-$Y"

68     Sep-2017
99     Nov-2017
105    Jan-2018
143    May-2018
144    Sep-2017
Name: settlement_date, dtype: object

In [7]:
# Specify the correct format: %b for abbreviated month name, %Y for 4-digit year
date_format = "%b-%Y"

# Convert date columns efficiently using the specified format
accepted_df_copy[date_columns] = accepted_df_copy[date_columns].apply(lambda col: pd.to_datetime(col,format =  date_format, errors = 'coerce'))

print(accepted_df_copy.issue_d.head())

0   2015-12-01
1   2015-12-01
2   2015-12-01
3   2015-12-01
4   2015-12-01
Name: issue_d, dtype: datetime64[ns]


In [8]:
# Check if all date columns are in date format
accepted_df_copy[date_columns].dtypes

issue_d                      datetime64[ns]
earliest_cr_line             datetime64[ns]
last_pymnt_d                 datetime64[ns]
next_pymnt_d                 datetime64[ns]
last_credit_pull_d           datetime64[ns]
sec_app_earliest_cr_line     datetime64[ns]
hardship_start_date          datetime64[ns]
hardship_end_date            datetime64[ns]
payment_plan_start_date      datetime64[ns]
debt_settlement_flag_date    datetime64[ns]
settlement_date              datetime64[ns]
dtype: object

In [9]:
# Check format of date columns
print(accepted_df_copy.settlement_date.dropna().head())

68    2017-09-01
99    2017-11-01
105   2018-01-01
143   2018-05-01
144   2017-09-01
Name: settlement_date, dtype: datetime64[ns]


### Convert categorical columns to 'category' type

In [10]:
# Convert Categorical columns to category

categorical_columns = ['term', 'grade', 'sub_grade', 'emp_length', 'home_ownership',
                       'verification_status', 'loan_status', 'pymnt_plan', 'purpose'
                       ,'addr_state', 'initial_list_status', 'application_type',
                       'hardship_flag', 'hardship_type', 'hardship_reason', 'hardship_status',
                       'hardship_loan_status', 'disbursement_method', 'debt_settlement_flag',
                       'settlement_status']

accepted_df_copy[categorical_columns] = accepted_df_copy[categorical_columns].apply(lambda col: col.astype('category'))

In [11]:
# Check if categorical columns are of categorical data type

accepted_df_copy[categorical_columns].dtypes

term                    category
grade                   category
sub_grade               category
emp_length              category
home_ownership          category
verification_status     category
loan_status             category
pymnt_plan              category
purpose                 category
addr_state              category
initial_list_status     category
application_type        category
hardship_flag           category
hardship_type           category
hardship_reason         category
hardship_status         category
hardship_loan_status    category
disbursement_method     category
debt_settlement_flag    category
settlement_status       category
dtype: object

In [12]:
# the title column will require some cleaning because the categories appear in a messiy format so I am converting 
# back to a string

'''
['\tcredit_card', '\tdebt_consolidation', '\thouse', '\tother', ..., 'zxcvb', '~Life Reorganization~', 
'~Summer Fun~', 'îîMY FIRST CAR îî']

'''

accepted_df_copy['title']

0          Debt consolidation
1                    Business
2                         NaN
3          Debt consolidation
4              Major purchase
                  ...        
2260696                   NaN
2260697    Debt consolidation
2260698                   NaN
2260699                   NaN
2260700                   NaN
Name: title, Length: 2260701, dtype: object

### Convert 'id' column to int

In [13]:
#Convert the id column to int

#check if 'id' column has any missing values
print(accepted_df_copy['id'].describe())

#convert id column from objext(text) to int

'''This code below didn't work because there is a row that is a string representing this value;
 'Total amount funded in policy code 1: 6417608175'
'''

# accepted_df_copy['id'] = accepted_df_copy['id'].astype(int)

count      2260701
unique     2260701
top       68407277
freq             1
Name: id, dtype: int64


"This code below didn't work because there is a row that is a string representing this value;\n 'Total amount funded in policy code 1: 6417608175'\n"

In [14]:
# find problematic row in 'id' column stopping it from being an int column
problematic_row = accepted_df_copy[accepted_df_copy['id'] == 'Total amount funded in policy code 1: 6417608175'].index

# Remove problematics row
accepted_df_copy = accepted_df_copy.drop(index = problematic_row[0])

In [15]:
# identify all rows in 'id' column that are not of type int

non_int_rows = accepted_df_copy[~accepted_df_copy['id'].apply(lambda x: str(x).isdigit())] #33 rows

print(len(non_int_rows['id']), 'non-int rows identified')
print('\n')
print(non_int_rows['id'])

32 non-int rows identified


421096     Total amount funded in policy code 2: 1944088810
528961     Total amount funded in policy code 1: 1741781700
528962      Total amount funded in policy code 2: 564202131
651664     Total amount funded in policy code 1: 1791201400
651665      Total amount funded in policy code 2: 651669342
749520     Total amount funded in policy code 1: 1443412975
749521      Total amount funded in policy code 2: 511988838
877716     Total amount funded in policy code 1: 2063142975
877717      Total amount funded in policy code 2: 823319310
983169     Total amount funded in policy code 1: 1538432075
983170      Total amount funded in policy code 2: 608903141
1117058    Total amount funded in policy code 1: 2087217200
1117059     Total amount funded in policy code 2: 662815446
1352689    Total amount funded in policy code 1: 3503840175
1352690     Total amount funded in policy code 2: 873652739
1481103    Total amount funded in policy code 1: 2050909275
1481104    

In [16]:
# check if all columns associated with the non_int_rows in 'id' column are empty

col_not_empty = non_int_rows.drop(columns=['id']).isna().all(axis=1)

if (col_not_empty.all() == True):
    print('All columns associated with non-int rows in \'id\' columns are empty')
else:
    print('All columns associated with non-int rows in \'id\' columns are\'nt empty')

All columns associated with non-int rows in 'id' columns are empty


In [17]:
# Drop all non-int rows in id column

#Get shape of accepted_df Dataframe before dropping rows
print('accepted_df shape before removing rows:', accepted_df_copy.shape)


# Step 1 - Get index of non-int rows in 'id' column to drop
nonint_rows_todrop = non_int_rows.index

print()
print(nonint_rows_todrop)
print(len(nonint_rows_todrop), 'rows to remove\n\n')

# Step 2 - remove non-int rows in 'id' column from accepted_df_copy

accepted_df_copy = accepted_df_copy.drop(index = nonint_rows_todrop)
print(len(nonint_rows_todrop), 'successfully dropped\n\n')

# Step 3 - Check accepted_df shape to confirm if 32 rows were dropped

print('accepted_df shape after removing rows:', accepted_df_copy.shape)


accepted_df shape before removing rows: (2260700, 151)

Index([ 421096,  528961,  528962,  651664,  651665,  749520,  749521,  877716,
        877717,  983169,  983170, 1117058, 1117059, 1352689, 1352690, 1481103,
       1481104, 1611877, 1611878, 1651665, 1654415, 1654416, 1751196, 1751197,
       1939379, 1939380, 2038501, 2038502, 2157151, 2157152, 2260699, 2260700],
      dtype='int64')
32 rows to remove


32 successfully dropped


accepted_df shape after removing rows: (2260668, 151)


In [18]:
# Check if we still have non-int rows in 'id' column

accepted_df_copy[~accepted_df_copy['id'].apply(lambda row: str(row).isdigit())] #Should return an empty DF

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term


In [19]:
# Soooo now we can finally Convert 'id' column to integer
accepted_df_copy['id'] = accepted_df_copy['id'].astype(int)

In [20]:
accepted_df_copy['id'].info()

<class 'pandas.core.series.Series'>
Index: 2260668 entries, 0 to 2260698
Series name: id
Non-Null Count    Dtype
--------------    -----
2260668 non-null  int64
dtypes: int64(1)
memory usage: 34.5 MB


### Check for other variables of type 'object' so we can convert to the right type if necessary

In [21]:
#Get data types of all variables and convert it to a DataFrame
new_data_types = pd.DataFrame(accepted_df_copy.dtypes)

# Reset index of Dataframe
new_data_types = new_data_types.reset_index()

#define column names
new_data_types.columns = ['variable', 'type']

#Get variables that are type 'object'
object_vars = new_data_types[new_data_types['type'] == 'object']

#Get all types
#new_data_types['type'] = new_data_types['type'].astype('category')

print(object_vars)


                     variable    type
10                  emp_title  object
18                        url  object
19                       desc  object
21                      title  object
22                   zip_code  object
59  verification_status_joint  object


In [22]:
#print(new_data_types['type'].cat.categories)

### Convert emp_title to categorical variable and remove leading whitespaces from category names

In [23]:
# Convert emp_title to category or leave as oject

accepted_df_copy['emp_title'] = accepted_df_copy['emp_title'].astype('category')

#Check type of emp_title
accepted_df_copy['emp_title'].cat.categories

Index(['\tCFO', '\tMultimedia Supervisor', '\tSlot technician',
       '\tVP - Operations', ' ', ' \tASR II', ' \tAdv Mtr Proj Fld Rep',
       ' \tAuto Body Repair', ' \tDriver', ' \tEmployee Strategies Manager',
       ...
       'zueck transportation', 'zulily', '{Owner}Truck Driver',
       '| Principal Business Solution Architect|',
       'År.  Technical Illustrator', '​Associate Tech Support Analyst',
       '​Financial Analyst', '​License Compliance Investigator',
       '​Senior IT Field Support', '👨‍🍳 '],
      dtype='object', length=512694)

In [24]:
#Export Categories to CSV file

file_path = '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/emp_title_categories.csv'

categories = accepted_df_copy['emp_title'].cat.categories

pd.DataFrame({'job_titles': categories}).to_csv(file_path, index = False)

From a quick overview of the CSV file I saw:

1.) numbers as job_titles
2.) job titles containing emojis
3.) duplicate Job_titles with spelling or formating differences
4.) text like this 'ÄãFinancial Analyst'
5.) Leading white spaces represented by '\t'

In [25]:
# Convert 'emp_title' back to Object in Panadas

accepted_df_copy['emp_title'] = accepted_df_copy['emp_title'].astype(str)

In [26]:
# Remove Leadning White spaces
accepted_df_copy['emp_title'] = accepted_df_copy['emp_title'].str.strip()

print(accepted_df_copy['emp_title'].head(10))

0                                    leadman
1                                   Engineer
2                               truck driver
3                Information Systems Officer
4                        Contract Specialist
5                       Veterinary Tecnician
6    Vice President of Recruiting Operations
7                                road driver
8                            SERVICE MANAGER
9                             Vendor liaison
Name: emp_title, dtype: object


In [27]:
accepted_df_copy['emp_title'].dtype

dtype('O')

In [28]:
accepted_df_copy['emp_title'].info()

<class 'pandas.core.series.Series'>
Index: 2260668 entries, 0 to 2260698
Series name: emp_title
Non-Null Count    Dtype 
--------------    ----- 
2260668 non-null  object
dtypes: object(1)
memory usage: 34.5+ MB


In [29]:
print(type(categories))

<class 'pandas.core.indexes.base.Index'>


### Removing numbers that are job titles

In [30]:
#all job_titles as numbers in emp_title

number_job_titles = [title for title in categories if str(title).isdigit() == True]

print(number_job_titles)

print('\nThere are ', len(number_job_titles), 'job titles as numbers')

['0132', '03', '0301', '0321', '04', '1', '10', '1052', '1099', '11', '112', '12', '121000', '13', '14', '1400', '14000', '15', '15000', '16', '17', '1811', '19', '2', '20', '21', '22', '25', '25312', '3', '30', '30005', '30304', '36000', '37', '38645', '4', '45000', '48000', '5', '553742017', '59106', '6', '6048', '7', '8', '9', '911', '9652651']

There are  49 job titles as numbers


In [31]:
#Check frequency of numeric Job_title to check if they are important

numeric_titles = accepted_df_copy[accepted_df_copy['emp_title'].str.isdigit()]

print(numeric_titles['emp_title'].value_counts())

emp_title
1            13
3            12
8             9
5             8
12            7
10            7
11            6
4             4
2             4
911           4
20            3
15            3
14            3
6             3
21            2
17            2
112           2
16            2
7             1
1052          1
0132          1
04            1
25312         1
30304         1
1811          1
9652651       1
30            1
553742017     1
121000        1
1400          1
36000         1
37            1
1099          1
30005         1
59106         1
03            1
38645         1
19            1
6153          1
15000         1
9             1
25            1
45000         1
48000         1
0321          1
22            1
14000         1
6048          1
0301          1
13            1
Name: count, dtype: int64


In [32]:
#Remove numeric titles expect for 911 and 112

import re
keep_titles = ['911', '112']


def remove_numeric_job_tiles(title):
    
    if pd.isna(title): #Handle Missing Values to avoid errors
        return False
    
    #Covert title to string
    title_str = str(title).strip()
    
    # Check if record is '911' or '112'
    if title_str in keep_titles:
        return True
    
    # Check job titles that have numbers in them e.g. '2nd pressman'
    if re.search(r'\d+', title_str) and not title_str.isdigit():
        return True 
    
    return not title_str.isdigit()


# Apply remove_numeric_job_tiles function
accepted_df_copy = accepted_df_copy[accepted_df_copy['emp_title'].apply(remove_numeric_job_tiles)]

In [33]:
#Check if we only have 6 numeric titles  
# - Recall we had six '911'titles and two '112' titles

print(accepted_df_copy['emp_title'][accepted_df_copy['emp_title'].str.isdigit() == True])

16225      911
116912     112
755033     911
900893     911
1465253    112
1696755    911
Name: emp_title, dtype: object


In [34]:
#Normalize Titles

#Make a copy of the orginal dataframe using 'accepted_df.copy' to avoid editing parent DF'
accepted_df_copy.loc[:,'emp_title'] = accepted_df_copy['emp_title'].str.lower()

In [ ]:
len(unique_titles)

In [39]:
import json
from rapidfuzz import fuzz, process
from tqdm import tqdm

def find_similar_titles_sorted(df, column_name, chunk_size=1000, similarity_threshold=90):
    """
    Identify job titles that are similar but contain spelling errors using sorted sampling.

    Args:
        df (pd.DataFrame): The DataFrame containing the dataset.
        column_name (str): The column containing job titles to analyze.
        chunk_size (int): The size of chunks to process for analysis.
        similarity_threshold (int): The threshold for considering titles as similar.

    Returns:
        dict: A dictionary of similar job titles.
    """
    # Get unique job titles and sort them alphabetically
    unique_titles = sorted(df[column_name].unique())

    # Split the sorted titles into chunks
    title_chunks = [unique_titles[i:i + chunk_size] for i in range(0, len(unique_titles), chunk_size)]

    # Initialize a single progress bar for all chunks
    total_titles = len(unique_titles)
    similar_titles = {}
    with tqdm(total=total_titles, desc="Processing Titles") as pbar:
        for chunk in title_chunks:
            for title in chunk:
                matches = process.extract(title, chunk, scorer=fuzz.ratio, limit=5)
                similar_titles[title] = [match[0] for match in matches if match[1] >= similarity_threshold and match[0] != title]
                pbar.update(1)

    # Filter out job titles with no close matches
    similar_titles = {key: value for key, value in similar_titles.items() if value}

    return similar_titles

# Example usage
similar_titles = find_similar_titles_sorted(accepted_df_copy, 'emp_title', chunk_size=1000, similarity_threshold=90)
with open('similar_titles_sorted.json', 'w') as f:
     json.dump(similar_titles, f)

Processing Titles: 100%|█████████████| 411763/411763 [00:17<00:00, 23296.00it/s]


In [41]:
# Display the results
for title, matches in similar_titles.items():
    print(f"'{title}' is similar to: {matches}")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



'underwriter support' is similar to: ['underwriter supporter']
'underwriter supporter' is similar to: ['underwriter support']
'underwriter tech' is similar to: ['underwriter teck']
'underwriter teck' is similar to: ['underwriter tech']
'underwriter v' is similar to: ['underwriter iv', 'underwriter 1', 'underwriter 2', 'underwriter 3']
'underwriter's assitant' is similar to: ['underwriters assistant', 'underwriter assistant', 'under writer assistant']
'underwriter/analyst' is similar to: ['underwriter analyst']
'underwriter/broker' is similar to: ['underwriter / broker']
'underwriterii' is similar to: ['underwriter ii', 'underwriteriii', 'underwriter iii', 'underwritter ii']
'underwriteriii' is similar to: ['underwriter iii', 'underwriterii', 'underwriter ii']
'underwriters' is similar to: ['underwriter', 'under writer', 'underwriteer', 'underwritier']
'underwriters assistant' is similar to: ['underwriter assistant', 'under writer assistant', "underwriter's assitant", 'underwriter assis

In [44]:
accepted_df_copy['emp_title'].value_counts()[:50]

emp_title
nan                         166969
teacher                      48460
manager                      45852
owner                        33591
registered nurse             23354
supervisor                   22306
driver                       22267
sales                        18984
rn                           17196
office manager               14231
project manager              13842
general manager              13317
truck driver                 12797
director                     10595
president                     9826
engineer                      8978
sales manager                 8532
operations manager            8183
police officer                7675
vice president                7625
technician                    7437
nurse                         7358
store manager                 7265
administrative assistant      7213
accountant                    6891
mechanic                      6548
attorney                      6375
account manager               6285
assistant 

In [42]:
import json

# Define the file path
file_path = '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/similar_titles.json'

# Save the similar titles to the JSON file
with open(file_path, 'w') as file:
    json.dump(similar_titles, file, indent=4)

print(f"Similar titles saved to '{file_path}'")


Similar titles saved to '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/similar_titles.json'


In [309]:
# Convert emp_title back to object type so we can remove \t
accepted_df_copy['emp_title'] = accepted_df_copy['emp_title'].astype(str)

# Remove white spaces
accepted_df_copy['emp_title'] = accepted_df_copy['emp_title'].str.strip()

# Convert 'emp_title' column to category type
accepted_df_copy['emp_title'] = accepted_df_copy['emp_title'].astype('category')

# Category types
categories = accepted_df_copy['emp_title'].cat.categories

#Save as csv

file_path = '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/emp_title_categories.csv'

pd.DataFrame({'job_titles':categories}).to_csv(file_path, index = False)

### Change 'url' column to the right datatype or drop it if not needed

In [45]:
#Inspect url Column
pd.set_option('display.max_colwidth', None) #make sure pandas doesn't truncate the url column


accepted_df_copy[['id', 'url']].head(5)

,id,url
0,68407277,https://lendingclub.com/browse/loanDetail.action?loan_id=68407277
1,68355089,https://lendingclub.com/browse/loanDetail.action?loan_id=68355089
2,68341763,https://lendingclub.com/browse/loanDetail.action?loan_id=68341763
3,66310712,https://lendingclub.com/browse/loanDetail.action?loan_id=66310712
4,68476807,https://lendingclub.com/browse/loanDetail.action?loan_id=68476807


Seems like the 'url' gets info for each specific 'id'. 

Let's check if the id from the id column matches the id in
in the url

In [46]:
# Extract the loan_id from the url using a regular expression
extracted_id = accepted_df_copy['url'].str.extract(r'loan_id=(\d+)')

In [47]:
#Check if the id was extracted correctly from the url
print(extracted_id.head())

# Check type of extracted_id
print(type(extracted_id[0]))

          0
0  68407277
1  68355089
2  68341763
3  66310712
4  68476807
<class 'pandas.core.series.Series'>


#### Convert extracted_id to same datatype as 'id'


In [48]:
#Check dtype of id column
print('Type of \'id\' column is', accepted_df_copy['id'].dtype)

#Change dtype of extracted_id columns to 'int'
extracted_id[0] = extracted_id[0].astype('int')

#Check dtype of extracted_id column
print('Type of \'extracted_id\' is', extracted_id[0].dtype)

Type of 'id' column is int64
Type of 'extracted_id' is int64


In [49]:
#Check if all id's between 'extracted_id' column and 'id' column match

all_match = (accepted_df_copy['id'] == extracted_id[0]).all()

print(all_match)

True


#### We can drop url column since there no mismatches

In [50]:
# Shape before dropping 'url'
print('Shape before drop: ', accepted_df_copy.shape)

Shape before drop:  (2260548, 151)


In [51]:
# Drop 'url' column
accepted_df_copy = accepted_df_copy.drop('url', axis = 1)

In [52]:
# Shape after dropping 'url'
print('Shape after drop: ', accepted_df_copy.shape)

Shape after drop:  (2260548, 150)


### Inspect 'desc' column and change 'desc' column to the right datatype if needed

'desc' contains information about the loan description wriiten by the borrower.

In [53]:
print(accepted_df_copy['desc'].head())

print(accepted_df_copy['desc'].dtype)


0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: desc, dtype: object
object


#### Notes from code above
From checking the data type of 'desc' and the first few rows it seems this column might be empty.

'desc' contains information about the loan description form the borrower.

We will rename this column to make it more intuitive to understand.

In [55]:
#Check if all values in the 'desc' column are all NaN

print(accepted_df_copy['desc'].info())

<class 'pandas.core.series.Series'>
Index: 2260548 entries, 0 to 2260698
Series name: desc
Non-Null Count   Dtype 
--------------   ----- 
126059 non-null  object
dtypes: object(1)
memory usage: 34.5+ MB
None


In [56]:
#count nuber of Null Values in 'desc' column

print(accepted_df_copy['desc'].isna().sum())

2134489


In [57]:
#Change name of column 'desc' to 'loan_purpose'
accepted_df_copy = accepted_df_copy.rename(columns = {'desc': 'loan_purpose'}) 

In [58]:
#Check if name change happened

accepted_df_copy['loan_purpose'].info()

<class 'pandas.core.series.Series'>
Index: 2260548 entries, 0 to 2260698
Series name: loan_purpose
Non-Null Count   Dtype 
--------------   ----- 
126059 non-null  object
dtypes: object(1)
memory usage: 34.5+ MB


In [392]:
accepted_df_copy['loan_purpose'].dropna().

2260694    nan
2260695    nan
2260696    nan
2260697    nan
2260698    nan
Name: loan_purpose, dtype: object

In [59]:
#Check for missing values in 'desc'/'loan_purpose'
print(accepted_df_copy['loan_purpose'].isna().sum())

2134489


In [394]:

accepted_df_copy['loan_purpose'].unique()

array(['nan',
       'We knew that using our credit cards to finance an adoption would squeeze us, but then medical and other unexpected expenses made the situation almost impossible. We are a stable family in a stable community. We just need to break a cycle of debt that is getting worse.',
       "I had a bad year two years ago, with some late and missed payments. I'm doing much better now, but I've got fees and some higher interest bits that have added up on top of the other stuff, and it's a little crazy. I'm hoping doing it thru Lending Club will make it easier - and cheaper - to pay off.",
       ...,
       'This loan will be used solely to consolidate credit card debts accrued while wife was/is unemployed.',
       'I have recently purchased and built a new home that I have always dreamed of having.  I would like to complete the project by putting a hottub in my backyard, however; I am not happy with the rate I  have been offered from GE to finance the spa.  I am paying cash fo

### Inspect 'title' column and change 'title' column to categorical data type if needed

In [358]:
print(accepted_df_copy['title'].info())

<class 'pandas.core.series.Series'>
Index: 2260668 entries, 0 to 2260698
Series name: title
Non-Null Count    Dtype 
--------------    ----- 
2260668 non-null  object
dtypes: object(1)
memory usage: 99.0+ MB
None


In [359]:
print(accepted_df_copy['title'].describe())

count                2260668
unique                 63155
top       Debt consolidation
freq                 1153293
Name: title, dtype: object


In [60]:
accepted_df_copy['title'].value_counts()[:20]

title
Debt consolidation           1153227
Credit card refinancing       469676
Home improvement              137428
Other                         127703
Major purchase                 44837
Medical expenses               25385
Business                       20800
Car financing                  20526
Debt Consolidation             15763
Vacation                       14443
Moving and relocation          13800
Home buying                    12713
Consolidation                   5385
debt consolidation              4837
Debt Consolidation Loan         3804
Credit Card Consolidation       2360
consolidation                   2175
Personal Loan                   2133
Consolidation Loan              1778
Home Improvement                1773
Name: count, dtype: int64

In [365]:
#Convert 'title' to categorical columns

#Strip white space before conversion
accepted_df_copy['title'] = accepted_df_copy['title'].str.strip()

#convert to categorical variable
accepted_df_copy['title'] = accepted_df_copy['title'].astype('category')

#show categories
print(accepted_df_copy['title'].cat.categories)

Index(['"CCC"',
       '"Discover" ing I no longer want to "Chase" after my own tail.',
       '"For those that said i couldn't"', '"Freedom!"',
       '"Funeral Expenses Loan', '"Get Out of Debt"',
       '"Get out of Debt" Consolidation Loan',
       '"Getting ahead on my debt" loan',
       '"Getting over the hump" debt consolidatn', '"Good Loan"',
       ...
       'zero dept', 'zero interest', 'zerodebt', 'zeusamoose', 'zipcar',
       'zonball Loan', 'zxcvb', '~Life Reorganization~', '~Summer Fun~',
       'îîMY FIRST CAR îî'],
      dtype='object', length=61456)


In [371]:
#Rename 'title' columns to 'loan_title'
accepted_df_copy = accepted_df_copy.rename(columns = {'title': 'loan_title'})

#Check name of the columns
print(accepted_df_copy['loan_title'].head())


0    Debt consolidation
1              Business
2               unknown
3    Debt consolidation
4        Major purchase
Name: loan_title, dtype: category
Categories (61456, object): ['"CCC"', '"Discover" ing I no longer want to "Chase" after my own tail.', '"For those that said i couldn't"', '"Freedom!"', ..., 'zxcvb', '~Life Reorganization~', '~Summer Fun~', 'îîMY FIRST CAR îî']


### Inspect 'zip_code' column and chnage to categorical variable if necessary

In [373]:
print(accepted_df_copy['zip_code'].head(10))

0    190xx
1    577xx
2    605xx
3    076xx
4    174xx
5    300xx
6    550xx
7    293xx
8    160xx
9    029xx
Name: zip_code, dtype: object


In [ ]:
['emp_title'].value_counts()[:20]

Seems like a portion of the zip_code was altered to preserve privacy of the user.

We will be just extracting the first 3 digits in another column called 'zip_prefix'

In [381]:
#Extract first 3 digits of the zip code
accepted_df_copy['zip_prefix'] = accepted_df_copy['zip_code'].str[:3]

#Check if first 3 digits seem to be extracted correctly
print(accepted_df_copy['zip_prefix'].head())

0    190
1    577
2    605
3    076
4    174
Name: zip_prefix, dtype: object


In [382]:
#Check if there is any x's in the extracted zip code
contains_x = accepted_df_copy['zip_prefix'].str.contains('x')

#count rows where 'x' is present
print(contains_x.sum())

0


In [380]:
accepted_df_copy['zip_prefix'].info()

count     2260667
unique        956
top           112
freq        23908
Name: zip_prefix, dtype: object

There are 956 unique zip_code prefixes so we can turn zip_prefix into a categorical variable

In [384]:
# Change zip_prefix to categorical variable

accepted_df_copy['zip_prefix'] = accepted_df_copy['zip_prefix'].astype('category')

#Check if we have 956 categories
print(accepted_df_copy['zip_prefix'].cat.categories)


Index(['007', '008', '009', '010', '011', '012', '013', '014', '015', '016',
       ...
       '990', '991', '992', '993', '994', '995', '996', '997', '998', '999'],
      dtype='object', length=956)
